In [ ]:
import asyncio
import websockets
import nest_asyncio
import pyaudio

# Fix for Jupyter Notebook
nest_asyncio.apply()

# Server settings
HOST = "0.0.0.0"
PORT = 8000

# Audio settings
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 44100  # Ensure this matches the client settings
CHUNK = 1024

# Initialize PyAudio
audio = pyaudio.PyAudio()
stream = audio.open(format=FORMAT, channels=CHANNELS, rate=RATE, output=True, frames_per_buffer=CHUNK)

async def stream_audio(websocket, path=None):  
    """Receives audio stream from the client and plays it in real-time."""
    print("🎤 Connection established. Receiving and playing audio...")

    try:
        async for message in websocket:
            if message == b"KEEP_ALIVE":  # Ignore keep-alive messages
                continue
            
            print(f"📡 Received {len(message)} bytes of audio data")
            stream.write(message)  # Play received audio in real-time
            
    except websockets.exceptions.ConnectionClosedError:
        print("❌ Client disconnected.")
    except Exception as e:
        print(f"🔥 Server Error: {e}")

async def start_server():
    """Starts the WebSocket server and keeps it running."""
    server = await websockets.serve(stream_audio, HOST, PORT)
    print(f"🎤 WebSocket Server running at ws://{HOST}:{PORT}")
    await server.wait_closed()  

# ✅ Properly start the server in Jupyter
loop = asyncio.get_event_loop()
if loop.is_running():
    print("🔄 Event loop already running, creating background task...")
    asyncio.create_task(start_server())  
else:
    loop.run_until_complete(start_server())